Autre implementation:
https://github.com/stephenbeckr/L-BFGS-B-C/blob/master/src/subalgorithms.c

L'implementation d'un L-BFGS-B n'est pas triviale. Si l'article de fondateur est très didactique et explicite, le code de référence, l'algorithme 778 est écrit en Fortran 77 et possède de nombreuses optimizations rendent sa compréhension, sa modification ou encore sa reproduction difficile. Pour des raisons de flexibilité, nous avons choisit une réimplementation en Python, s'appuyant sur numpy et scipy. Le code produit, bien que plus gourmand en mémoire, donne globallement les mêmes résultats (nombre d'évaluations de fonction, gradient et justesse des résultats). Ajouter les 10 cas tests.

Nous précisons ici deux éléments importants.
En particulier, deux éléments initialement non présent dans l'article original ne sont que vaguement évoqué dans les articles de mise à jour. La denrière ayant été faite en 2011, portant le code dans sa version 3.0.

L'intégralité du code est facile à comprendre. Il n'y a qu'un point un peu plus compliqué, et nous le détaillons ici:

# Optimization 1

La première otpimization intervient dans la recherche des points de cauchy généralisés. Dans la section 4 de [1], Le calcul de la dérivée seconde $f''$ fait intervenir la matrice $\mathbf{M}$ 

$$
f'' = \theta \mathbf{d}^{\mathrm{T}}\mathbf{d} - \mathbf{d}^{\mathrm{T}}\mathbf{WMW}^{T}\mathbf{d}
$$

En suivant la notation du papier mais en omettant l'indice $k$, la matrice $\mathbf{M}$ est définie par l'équation 3.4,  $\mathbf{M} = \begin{pmatrix} - \mathbf{D} & \mathbf{L}^{\mathrm{T}} \\ \mathbf{L} & \theta \mathbf{S}_{T}\mathbf{S} \end{pmatrix}^{-1}$ et a une dimension (2m, 2m), m étant le nombre de corrections apportées à l'approximation du Hessien. Cette matrice n'est pas définie positive mais ces quatres blocks le sont. On peut donc effectuer une factorization de cholesky à partir des sous blocks.


However, the factorization is not explained. Thus, we would like to provide some details about how to achieve it. Recall the $\mathbf{LDL}^{\mathrm{T}}$ factorization for a block matrix:

$$\mathbf{K} = \begin{pmatrix} \mathbf{K}_{11} & \mathbf{K}_{21}^{\mathrm{T}} \\ \mathbf{K}_{21} & \mathbf{K}_{22} \end{pmatrix}
 =\begin{pmatrix} 1 & 0 \\ \mathbf{K}_{21} \mathbf{K}_{11}^{-1} & 1 \end{pmatrix}
  \begin{pmatrix} \mathbf{K}_{11} & 0 \\ 0 & \mathbf{P} \end{pmatrix}
  \begin{pmatrix} 1 & \mathbf{K}_{11}^{-1} \mathbf{K}_{21}^{\mathrm{T}} \\ 0 & 1 \end{pmatrix}$$
where $\mathbf{P} = \mathbf{K}_{22} - \mathbf{K}_{21} \mathbf{K}_{11}^{-1} \mathbf{K}_{21}^{\mathrm{T}}$ is the Schur complement. Clearly one need $\mathbf{K}_{11}^{-1}$ and $\mathbf{P}^{-1}$ to solve using this factorization.

Substituting $\mathbf{K}_{11} = \mathbf{L}_{11} \mathbf{L}_{11}^{\mathrm{T}}$ and $\mathbf{P} = \mathbf{L}_P \mathbf{L}_P^{\mathrm{T}}$ into the factorization above, we obtain the Cholesky factorization [1]:
$$\begin{aligned}
\mathbf{K} & =\begin{pmatrix}
\mathbb{1} & 0\\
\mathbf{K}_{21} \left( \mathbf{L}_{11} \mathbf{L}_{11}^{\mathrm{T}}\right)^{-1} & \mathbb{1}
\end{pmatrix}\begin{pmatrix}
\mathbf{L}_{11} \mathbf{L}_{11}^{\mathrm{T}} & 0\\
0 & \mathbf{L}_P \mathbf{L}_P^{\mathrm{T}}
\end{pmatrix}\begin{pmatrix}
\mathbb{1} & \left( \mathbf{L}_{11} \mathbf{L}_{11}^{\mathrm{T}}\right)^{-1} \mathbf{K}_{21}^{\mathrm{T}}\\
0 & \mathbb{1}
\end{pmatrix}\\
 & =\left(\begin{pmatrix}
\mathbb{1} & 0\\
\mathbf{K}_{21}\left( \mathbf{L}_{11} \mathbf{L}_{11}^{\mathrm{T}}\right)^{-1} & \mathbb{1}
\end{pmatrix}\begin{pmatrix}
\mathbf{L}_{11} & 0\\
0 & \mathbf{L}_P
\end{pmatrix}\right)\left(\begin{pmatrix}
\mathbf{L}_{11}^{\mathrm{T}} & 0\\
0 & \mathbf{L}_P^{\mathrm{T}}
\end{pmatrix}\begin{pmatrix}
\mathbb{1} & \left( \mathbf{L}_{11} \mathbf{L}_{11}^{\mathrm{T}}\right)^{-1} \mathbf{K}_{21}^{\mathrm{T}}\\
0 & \mathbb{1}
\end{pmatrix}\right)\\
 & =\begin{pmatrix}
\mathbf{L}_{11} & 0\\
\mathbf{K}_{21}\mathbf{L}_{11}^{-T} & \mathbf{L}_P
\end{pmatrix}\begin{pmatrix}
\mathbf{L}_{11}^{\mathrm{T}} & \mathbf{L}_{11}^{-1} \mathbf{K}_{21}^{\mathrm{T}}\\
0 & \mathbf{L}_P^{\mathrm{T}}
\end{pmatrix} = \mathbf{L}_{K} \mathbf{L}_{K}^{\mathrm{T}} .
\end{aligned}
$$

As a consequence, $\mathbf{D}$ being a diagonal matrix, $M$ can be factorized as:
$$

\mathbf{M} = \mathbf{L}_{M} \mathbf{L}_{M}^{\mathrm{T}} = \begin{pmatrix}
\mathbf{D}^{1/2} & 0\\
-\mathbf{L}_{21}\mathbf{D}^{-1/2} & \mathbf{J}
\end{pmatrix}\begin{pmatrix}
-\mathbf{D}^{1/2} & \mathbf{D}^{-1/2} \mathbf{L}_{21}^{\mathrm{T}}\\
0 & \mathbf{J}^{\mathrm{T}}
\end{pmatrix} = \mathbf{L}_{K} \mathbf{L}_{K}^{\mathrm{T}}

$$

with $\mathbf{L}_{21}$ the lower triangle of the $\theta \mathbf{S}_{T}\mathbf{S}$ cholesky factorization. And $\mathbf{J}$ the lower triangle of $\mathbf{JJ}^{\mathrm{T}} = \theta \mathbf{S}_{T}\mathbf{S} - \mathbf{LD}^{-1} \mathbf{L}^{\mathrm{T}}$ also obtained by cholesky factorization.
Then all operations involving a product with $\mathbf{M}$ can be operated very easily by solving the linear system $Ax = B$ as previously explained in "ADD REF".

# Optimization 2

Section 5 of Byrd et al. [1995] describes three methods for performing the subspace minimization: direct primal, primal CG, and dual. 
As explained by Zhu et al. (1997), "primal and dual can be implemented in a unified framework in which they are very similar; they require essentially the same amount of computation and perform equally well in practice". As a consequence, the current L-BFGS-B code (887) uses only the primal method for subspace minimization. However, the resolution differs a bit from the original paper (recipe given in pages 11-12). Indeed, Zhu et al. 1997, Zhu et al (1997) explain that in eq . 5.11

$$
    \hat{\mathbf{d}}^{u} = \dfrac{1}{\theta}\hat{\mathbf{r}}^{c}+\dfrac{1}{\theta^{2}}\mathbf{Z}^{\mathrm{T}}\mathbf{W}\left(\mathbf{I} - \dfrac{1}{\theta}\mathbf{MW}^{\mathrm{T}}\mathbf{ZZ}^{\mathrm{T}}\mathbf{W}\right)^{-1}\mathbf{M} \mathbf{W}^{\mathrm{T}}\mathbf{Z}\hat{\mathbf{r}}^{c}
$$

Can actually be written as 

$$
    \hat{\mathbf{d}}^{u} = \dfrac{1}{\theta}\hat{\mathbf{r}}^{c}+\dfrac{1}{\theta^{2}}\mathbf{Z}^{\mathrm{T}}\mathbf{W}\mathbf{K}^{-1} \mathbf{W}^{\mathrm{T}}\mathbf{Z}\hat{\mathbf{r}}^{c}
$$

where 

$$
    \mathbf{K}^{-1} = (\mathbf{I} - \dfrac{1}{\theta}\mathbf{MW}^{\mathrm{T}}\mathbf{ZZ}^{\mathrm{T}}\mathbf{W})^{-1}\mathbf{M} 

$$

and they remark that

$$
    \mathbf{K} = \begin{bmatrix} -\mathbf{D} - \dfrac{1}{\theta}\mathbf{Y}^{\mathrm{T}}\mathbf{ZZ}^{\mathrm{T}}\mathbf{Y}  & \mathbf{L}_{A}^{\mathrm{T}} - \mathbf{R}_{Z}^{\mathrm{T}} \\ \mathbf{L}_{A} - \mathbf{R}_{Z} & \theta \mathbf{S}^{\mathrm{T}}\mathbf{AA}^{\mathrm{T}}\mathbf{S} \end{bmatrix}
$$

where $\mathbf{L}_A$ is the strict lower triangle of $\mathbf{S}^{\mathrm{T}}\mathbf{AA}^{\mathrm{T}}\mathbf{S}$ and $\mathbf{R}_{Z}$ is the upper triangle of $\mathbf{Y}^{\mathrm{T}}\mathbf{ZZ}^{\mathrm{T}}\mathbf{Y}$. Although this matrix is not positive definite, they claim it can be factorized symmetrically by using Cholesky factorizations of the submatrices, and we do so in the L-BFGS-B code. However, the factorization is not explained. 

- Unlike the previous one, we did not find any report detailing the factorization. We give it here.
Thus, we would like to provide some details about how to achieve it.

L'approche est un peu différente du cas précédent, car le terme $\mathbf{D} + \dfrac{1}{\theta}\mathbf{Y}^{\mathrm{T}}\mathbf{ZZ}^{\mathrm{T}}\mathbf{Y}$ est défini positif et peut donc être factorisé, mais pas son opposé. La factorization sera donc de la forme $\mathbf{L}_K\mathbf{E}\mathbf{L}_K^{T}$ avec $\mathbf{E} = \begin{pmatrix} -\mathbf{I} & 0 \\ 0 & \mathbf{I} \end{pmatrix}$

En notant utilisant la notation finale voulue:

$$
\begin{aligned}
\mathbf{A} = \mathbf{L}_\mathrm{A}\mathbf{E}\mathbf{L}_{\mathrm{A}}^{T} &= \begin{pmatrix} \mathbf{L}_{11} & 0 \\ \mathbf{L}_{21} & \mathbf{L}_{22} \end{pmatrix} \begin{pmatrix} \mathbf{I} & \mathbf{0} \\ \mathbf{0} & -\mathbf{I} \end{pmatrix} \begin{pmatrix} \mathbf{L}_{11}^{\mathrm{T}} & \mathbf{L}_{21}^{\mathrm{T}} \\ \mathbf{0} & \mathbf{L}_{22}^{\mathrm{T}} \end{pmatrix}
\\
&= \begin{pmatrix} \mathbf{L}_{11} \mathbf{L}_{11}^{\mathrm{T}} & \mathbf{L}_{11} \mathbf{L}_{21}^{\mathrm{T}} \\ \mathbf{L}_{21} \mathbf{L}_{11}^{\mathrm{T}}  & \mathbf{L}_{21} \mathbf{L}_{21}^{\mathrm{T}} - \mathbf{L}_{22} \mathbf{L}_{22}^{\mathrm{T}} \end{pmatrix}^{\mathrm{T}}
\end{aligned}
$$

Par identification, on obtient alors en trois étapes successives:

- $\mathbf{L}_{11}$ the lower triangle of the cholesky factorization of $\mathbf{D} + \dfrac{1}{\theta}\mathbf{Y}^{\mathrm{T}}\mathbf{ZZ}^{\mathrm{T}}\mathbf{Y}$

- $\mathbf{L}_{21} = \left(\mathbf{L}_{11}^{-1} \left( \mathbf{R}_{Z} - \mathbf{L}_{A} \right)^{\mathrm{T}}\right)^{T}$ 

- $\mathbf{L}_{22}$ the lower triangle of the cholesky factorization of $\theta \mathbf{S}^{\mathrm{T}}\mathbf{AA}^{\mathrm{T}}\mathbf{S} + \mathbf{L}_{21} \mathbf{L}_{21}^{\mathrm{T}}$